DEEP REINFORCEMENT LEARNING EXPLAINED - 09
# **The Value Iteration Algorithm**
## Value Iteration in Practice

In [0]:
import gym
import collections
from torch.utils.tensorboard import SummaryWriter

ENV_NAME = "FrozenLake-v0"
#ENV_NAME = "FrozenLake8x8-v0"  
GAMMA = 0.9
TEST_EPISODES = 20
REWARD_GOAL = 0.8
N =100

### The Agent

In [0]:
class Agent:
    def __init__(self):
        self.env = gym.make(ENV_NAME)
        self.state = self.env.reset()
        self.rewards = collections.defaultdict(float)
        self.transits = collections.defaultdict(
            collections.Counter)
        self.values = collections.defaultdict(float)

    def play_n_random_steps(self, count):
        for _ in range(count):
            action = self.env.action_space.sample()
            new_state, reward, is_done, _ = self.env.step(action)
            self.rewards[(self.state, action, new_state)] = reward
            self.transits[(self.state, action)][new_state] += 1
            if is_done:
                self.state = self.env.reset() 
            else: 
                self.state = new_state

    def calc_action_value(self, state, action):
        target_counts = self.transits[(state, action)]
        total = sum(target_counts.values())
        action_value = 0.0
        for tgt_state, count in target_counts.items():
            reward = self.rewards[(state, action, tgt_state)]
            val = reward + GAMMA * self.values[tgt_state]
            action_value += (count / total) * val
        return action_value

    def select_action(self, state):
        best_action, best_value = None, None
        for action in range(self.env.action_space.n):
            action_value = self.calc_action_value(state, action)
            if best_value is None or best_value < action_value:
                best_value = action_value
                best_action = action
        return best_action


    def value_iteration(self):
        for state in range(self.env.observation_space.n):
            state_values = [
                self.calc_action_value(state, action)
                for action in range(self.env.action_space.n)
            ]
            self.values[state] = max(state_values)

### Train the Agent

In [12]:
test_env = gym.make(ENV_NAME)
agent = Agent()
writer = SummaryWriter()

iter_no = 0
best_reward = 0.0
 
while best_reward < REWARD_GOAL:
        
        agent.play_n_random_steps(N)

        agent.value_iteration()

        iter_no += 1
        reward_test = 0.0
        for _ in range(TEST_EPISODES):
            total_reward = 0.0
            state = test_env.reset()
            while True:
                action = agent.select_action(state)
                new_state, new_reward, is_done, _ = test_env.step(action)
                total_reward += new_reward
                if is_done: break
                state = new_state
            reward_test += total_reward
        reward_test /= TEST_EPISODES

        writer.add_scalar("reward", reward_test, iter_no)
        if reward_test > best_reward:
            print("Best reward updated %.2f at iteration %d " % (reward_test ,iter_no) )
            best_reward = reward_test

writer.close()

Best reward updated 0.10 at iteration 12 
Best reward updated 0.35 at iteration 14 
Best reward updated 0.60 at iteration 15 
Best reward updated 0.70 at iteration 24 
Best reward updated 0.75 at iteration 35 
Best reward updated 0.95 at iteration 36 


### Test the Agent

In [0]:
new_test_env = gym.make(ENV_NAME) 
state = new_test_env.reset()
new_test_env.render()
is_done = False
iter_no = 0
while not is_done:
     print (state)
     action = agent.select_action(state)
     new_state, reward, is_done, _ = new_test_env.step(action)
     test_env.render()
     state = new_state
     iter_no +=1
print("reward =    ", reward)
print("iterations =", iter_no)

In [0]:
%load_ext tensorboard


In [0]:
tensorboard  --logdir=runs